# Bayesian Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [1]:
# Add "beer" to the PYTHONPATH
import random
from collections import defaultdict
import sys
sys.path.insert(0, '../')


import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [2]:
torch.__version__

'0.4.1'

## Data

Generate some normally distributed data:

In [3]:
mean = np.zeros(2) 
cov = np.array([
    [2, .5],
    [.5, 1]
])
data = np.random.multivariate_normal(mean, cov, size=1000)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, line_color='black', fill_alpha=.3)

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [4]:
data_mean = torch.from_numpy(data.mean(axis=0)).float()
data_var = torch.from_numpy(data.var(axis=0)).float()

normal_iso = beer.Normal.create(data_mean, data_var, 1., cov_type='isotropic')
normal_diag = beer.Normal.create(data_mean, data_var, 1., cov_type='diagonal')
normal_full = beer.Normal.create(data_mean, data_var, 1., cov_type='full')

models = {
    'normal_full': normal_full,
    'normal_diag': normal_diag,
    'normal_iso': normal_iso
}

## Variational Bayes Training 

In [5]:
nbatches = 10
X = torch.from_numpy(data).float()
batches = X.view(nbatches, -1, 2)

In [6]:
epochs = 20
lrate = .1


optims = {
    model_name: beer.BayesianModelOptimizer(
        model.mean_field_groups, lrate)
    for model_name, model in models.items()
}

elbos = {
    model_name: [] 
    for model_name in models
}  


for epoch in range(epochs):
    for name, model in models.items():
        batch_ids = list(range(len(batches)))
        random.shuffle(batch_ids)
        for batch_id in batch_ids:
            batch = batches[batch_id]
            
            optim = optims[name]

            optim.init_step()
            elbo = beer.evidence_lower_bound(model, batch, datasize=len(X))
            elbo.backward()
            optim.step()

            elbos[name].append(float(elbo) / len(X))
        

In [7]:
colors = {
    'normal_iso': 'green',
    'normal_diag': 'blue',
    'normal_full': 'red',
    
}
# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
for model_name, elbo in elbos.items():
    fig.line(range(len(elbo)), elbo, legend=model_name, color=colors[model_name])
fig.legend.location = 'bottom_right'

show(fig)

In [8]:
models['normal_iso'].mean

tensor([-0.0038,  0.0370])

In [9]:
fig = figure(
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, normal_iso.mean.numpy(), normal_iso.cov.numpy(), 
                     line_color='black', fill_alpha=.3, color='green')
plotting.plot_normal(fig, normal_diag.mean.numpy(), normal_diag.cov.numpy(), 
                     line_color='black', fill_alpha=.3, color='#98AFC7')
plotting.plot_normal(fig, normal_full.mean.numpy(), normal_full.cov.numpy(), 
                     line_color='black', fill_alpha=.3, color='#C7B097')

show(fig)

In [31]:
data_mean = torch.from_numpy(data.mean(axis=0)).float()
data_var = torch.from_numpy(data.var(axis=0)).float()

normal_iso = beer.Normal.create(data_mean, data_var, 1., cov_type='isotropic')
normal_diag = beer.Normal.create(data_mean, data_var, 1., cov_type='diagonal')
normal_full = beer.Normal.create(data_mean, data_var, 1., cov_type='full')

models = {
    'normal_full': normal_full,
    'normal_diag': normal_diag,
    'normal_iso': normal_iso
}

model = models['normal_iso']

In [32]:
stats = model.sufficient_statistics(X)
model.marginal_log_likelihood(stats).sum()

tensor(-1961.7910)

In [33]:
iters = 20

batch_stats = {model_name: defaultdict(lambda: defaultdict(lambda: None))
               for model_name in models}
optims = {model_name: beer.CVBOptimizer(model.bayesian_parameters())
          for model_name, model in models.items()}
elbos = {model_name: [] for model_name in models}  

for epoch in range(epochs):
    batch_ids = list(range(len(batches)))
    random.shuffle(batch_ids)
    for name, model in models.items():
        for batch_id in batch_ids:
            optim = optims[name]
            optim.init_step(batch_stats[name][batch_id])
            elbo = beer.collapsed_evidence_lower_bound(model, batches[batch_id])
            batch_stats[name][batch_id] = elbo.backward()
            optim.step()
            elbos[name].append(float(elbo) / len(batches[batch_id]))

In [35]:
colors = {
    'normal_iso': 'green',
    'normal_diag': 'blue',
    'normal_full': 'red',
    
}
# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
for model_name, elbo in elbos.items():
    fig.line(range(len(elbo)), elbo, legend=model_name, color=colors[model_name])
fig.legend.location = 'bottom_right'

show(fig)

In [36]:
for name, model in models.items():
    stats = model.sufficient_statistics(X)
    mllh = model.marginal_log_likelihood(stats).sum()
    print(name, float(mllh))

normal_full -1338.126708984375
normal_diag -1343.39111328125
normal_iso -1404.96875


In [37]:
fig = figure(
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, normal_iso.mean.numpy(), normal_iso.cov.numpy(), 
                     line_color='black', fill_alpha=.3, color='green')
plotting.plot_normal(fig, normal_diag.mean.numpy(), normal_diag.cov.numpy(), 
                     line_color='black', fill_alpha=.3, color='#98AFC7')
plotting.plot_normal(fig, normal_full.mean.numpy(), normal_full.cov.numpy(), 
                     line_color='black', fill_alpha=.3, color='#C7B097')

show(fig)